<a href="https://colab.research.google.com/github/singaravelan/TensorFlow_programming/blob/main/Additional%20-%20Lets_compete_in_kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

In [ ]:
# Load the dataset
df = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')

# Drop unnecessary columns
df = df[['text', 'target']]

# Handle missing values if any
df = df.dropna()

# Split the data into training and testing sets
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
# Tokenize the text data
max_words = 10000  # Maximum number of words to keep in the vocabulary
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(train_data['text'])

# Convert text data to sequences
train_sequences = tokenizer.texts_to_sequences(train_data['text'])
test_sequences = tokenizer.texts_to_sequences(test_data['text'])

# Pad sequences to have consistent length
max_length = 20  # Adjust as needed
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post', truncating='post')
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding='post', truncating='post')

In [ ]:
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim,
                    embeddings_initializer=Constant(embedding_matrix),
                    input_length=max_length, trainable=False))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(64))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_padded, train_data['target'], epochs=5, validation_data=(test_padded, test_data['target']))

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(test_padded, test_data['target'])
print(f'Accuracy: {accuracy * 100:.2f}%')

In [ ]:
# Load the dataset
test_df = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

# Drop unnecessary columns
test_df_prediction = test_df['text']

new_sequences = tokenizer.texts_to_sequences(test_df_prediction)
new_padded = pad_sequences(new_sequences, maxlen=max_length, padding='post', truncating='post')

predictions = model.predict(new_padded)


In [ ]:
binary_predictions = (predictions > 0.5).astype(int)
print(binary_predictions)

In [ ]:
test_df["target"] = binary_predictions

In [ ]:
write = test_df[['id','target']]

In [ ]:
write.to_csv('submission.csv',index=False)